In [1]:
import datetime as dt
import pandas as pd

In [2]:
from private import Private

In [3]:
from st_etl_tools.etl_tools.fundamental.dart import DartFss

dart_fss = DartFss(Private.dart_api_key)

corps = dart_fss.load_corps(only_public=True)
corps

Output()

Output()

Output()

,corp_code,corp_name,stock_code,modify_date,sector,product,corp_cls,trading_halt,issue
2006,00260985,한빛네트,036720,20170630,NaN,NaN,NaN,NaN,NaN
2018,00264529,엔플렉스,040130,20170630,NaN,NaN,NaN,NaN,NaN
2019,00358545,동서정보기술,055000,20170630,NaN,NaN,NaN,NaN,NaN
2781,00231567,애드모바일,032600,20170630,NaN,NaN,NaN,NaN,NaN
3884,00247939,씨모스,037600,20170630,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
104138,00126414,삼성제약,001360,20231005,의약품 제조업,"의약품(까스명수,쓸기담,에프킬라에어졸,액체우황청심원) 제조,도매",Y,NaN,NaN
104148,00116426,코센,009730,20231205,1차 철강 제조업,"스테인리스파이프, 선박용배관제작",K,NaN,NaN
104149,00107987,남해화학,025860,20231205,"비료, 농약 및 살균, 살충제 제조업","비료(요소,복합비료),공업제품원료 제조,도매",Y,NaN,NaN
104150,01068658,디딤이앤에프,217620,20231206,음식점업,"직영음식점, 축산물가공, 프랜차이즈",K,NaN,NaN


In [4]:
from st_etl_tools.etl_tools.fundamental.dart_preproc import CorpPreProc

corp_preproc = CorpPreProc(corps)
preproc_corps = corp_preproc()
preproc_corps.to_csv("./data/corps.csv")

In [5]:
from st_etl_tools.etl_tools.fundamental.dart import DartExtractor

dart_extractor = DartExtractor(Private.dart_api_key)

reprt = "11011"
corp_codes = list(set(corps["corp_code"]))
year = "2023"

fundamentals_v1 = dart_extractor.load_fundamentals(corps=corp_codes, year=year, reprt=reprt)
left_corp_codes = list(set(corp_codes) - set(fundamentals_v1["corp_code"]))


year = "2022"

fundamentals_v2 = dart_extractor.load_fundamentals(corps=left_corp_codes, year=year, reprt=reprt)
fundamentals = pd.concat([fundamentals_v1, fundamentals_v2], axis=0)

In [6]:
from st_etl_tools.etl_tools.fundamental.dart_preproc import FundamentalPreProc

fundamental_preproc = FundamentalPreProc(fundamentals)

cfs_fundamental = fundamental_preproc("CFS")
cfs_fundamental.to_csv("./data/cfs_fundamental.csv")

ofs_fundamental = fundamental_preproc("OFS")
ofs_fundamental.to_csv("./data/ofs_fundamental.csv")

In [7]:
from st_etl_tools.etl_tools.technical.fdr import FdrExtractor

fdr_extractor = FdrExtractor()
stocks = fdr_extractor.load_stocks(only_major=True)

In [8]:
from st_etl_tools.etl_tools.technical.fdr_preproc import StockPreProc


stock_preproc = StockPreProc(stocks)
stocks = stock_preproc()
stocks.to_csv("./data/stocks.csv")

In [9]:
stock_codes = list(set(stocks["stock_code"]))
end_date = dt.date.today() - dt.timedelta(days=1)
start_date = end_date - dt.timedelta(days=365 * 5)

ohlcvs = fdr_extractor.load_ohlcvs(stock_codes, start_date, end_date)

  0%|          | 0/2623 [00:00<?, ?it/s]

100%|██████████| 2623/2623 [10:06<00:00,  4.33it/s]


In [10]:
from st_etl_tools.etl_tools.technical.fdr_preproc import OhlcvPreProc

ohlcv_preproc = OhlcvPreProc(ohlcvs)
ohlcvs = ohlcv_preproc()
ohlcvs.to_csv("./data/ohlcvs.csv")